## Guide to Annotation and Continuous Learning with NVIDIA MONAI Cloud APIs

In this guide, we delve deep into the process of annotation and continuous learning using NVIDIA MONAI Cloud APIs. As the bedrock of medical imaging, accurate annotations are pivotal, and the continuous refinement of models ensures they deliver the best results over time. We'll walk through the various steps and considerations involved in this process.

## Table of Contents

- Dataset and Experiment Setup
- Configuring Annotation and Continuous Learning Parameters
- VISTA Workflows
- Annotation Workflow
- Stopping a Continuous Learning Job
- Exporting the Experiment
- Conclusion

## Introduction

Annotation and Continuous Learning are core features of NVIDIA MONAI Cloud APIs, streamlining the process of refining datasets and enhancing model performance over time. Continuous learning leverages accumulated annotations to improve the model iteratively. This guide will walk you through the process of setting up and optimizing these tasks.

## Dataset and Experiment Setup

Before diving into annotation and continuous learning, we're going to quickly create our dataset and experiment that will be used for the annotation workflow.  

**Note:** We're going to use the `realtime_infer` parameter when creating our experiment as that will automatically load the experiment and make sure it's ready for our annotation and continuous learning workflow.

We've covered these steps in-depth in our other notebooks, you can find them below. If you haven't already gone through those notebooks, we encourge you to go back and review those first.

- [Generating and Managing Your Credentials](./Generating%20and%20Managing%20Your%20Credentials.ipynb)
- [Dataset Creation and Experiment Selection](./Dataset%20Creation%20and%20Experiment%20Selection.ipynb)

In [ ]:
import json
import requests
import time

In [ ]:
# API Endpoint and Credentials
host_url = "<MONAI Cloud API URL>"
ngc_api_key = "<NGC API Key>"
# Dicom Server
dicom_web_endpoint = "<DICOMWeb address>" # For example "http://127.0.0.1:8042/dicom-web".
dicom_client_id = "<DICOMWeb user ID>"    # If Authentication is enabled, then provide username
dicom_client_secret = "<DICOMWeb secret>" # If Authentication is enabled, then provide password

In [ ]:
# NGC UID 
data = json.dumps({"ngc_api_key": ngc_api_key})
response = requests.post(f"{host_url}/api/v1/login", data=data)
print(response.status_code)
assert response.status_code == 201, f"Login failed, got status code: {response.status_code}."
assert "user_id" in response.json().keys(), "user_id is not in response."
user_id = response.json()["user_id"]
print("User ID",user_id)
assert "token" in response.json().keys(), "token is not in response."
token = response.json()["token"]
print("JWT",token)

# Construct the URL and Headers
base_url = f"{host_url}/api/v1/users/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

# MLFlow server
use_mlflow = False
mlflow_server_address = "" # For example "http://127.0.0.1:5000".
mlflow_experiment_name = "" # For example "my_experiment"

data = {
    "name": "mydataset",
    "description":"a demo dataset",
    "type": "semantic_segmentation",
    "format": "monai",
    "client_url": f"{dicom_web_endpoint}",
    "client_id": f"{dicom_client_id}",
    "client_secret": f"{dicom_client_secret}",
}

endpoint = f"{base_url}/datasets"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Create dataset failed, got {response.json()}."
res = response.json()
dataset_id = res["id"]
print("Dataset creation succeeded with dataset ID: ", dataset_id)
print("---------------------------------\n")
print(json.dumps(res, indent=2))
    
endpoint = f"{base_url}/experiments"
response = requests.get(endpoint, headers=headers)
assert response.status_code == 200, f"List Base Experiment failed, got {response.json()}."
res = response.json()

# VISTA-3D
ptm_vista = [p for p in res if p["network_arch"] == "monai_vista3d" and not len(p["base_experiment"])][0]["id"]
print(f"Base Experiment ID for VISTA Experiment: {ptm_vista}")
    
data = {
  "name": "my_vista",
  "description": "based on vista",
  "network_arch": "monai_vista3d",
  "base_experiment": [ ptm_vista ],
  "inference_dataset": dataset_id,
  "eval_dataset": dataset_id,
  "train_datasets": [ dataset_id ],
  "realtime_infer": True, # Auto loads MONAI bundle and enables real-time inference
  "model_params":{
      "labels": {
            "1": "liver",
            "2": "kidney",
            "3": "spleen",
            "4": "pancreas",
            "5": "right kidney"
    }
  }
}

endpoint = f"{base_url}/experiments"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Create experiment failed, got {response.json()}."
res = response.json()
experiment_id = res["id"]
model_network = res["network_arch"]
print("Experiment creation succeeded with experiment ID:", experiment_id)
print("---------------------------------\n")
print(json.dumps(res, indent=2))

## Configuring Annotation and Continuous Learning Parameters

Continuous learning is the backbone of keeping our models accurate and up-to-date. As new data is annotated, the model has the potential to learn and adapt. However, to kick off this process, we need to specify certain parameters that inform the system how and when to refine the model.

With this job, the model will be fine tuned with new labeled samples after several notifications. A fine tuned model can generate a better annotation results, therefore improving the annotation efficiency.

*If you prefer to only annotate data without the continuous learning process, you can simply skip this step. You can still use the annotation tools and workflows outlined in the upcoming sections independently.*

### API Call for Continuous Learning Job

Below, we provide the API call needed to create a continuous learning job for a model:

In [ ]:
train_spec = {
    "epochs": 10,
    "val_interval": 1,
}


if use_mlflow:
    mlflow_spec = {
        "tracking": "mlflow",
        "tracking_uri": f"{mlflow_server_address}",
        "experiment_name": f"{mlflow_experiment_name}",
        "train#handlers#-1#artifacts": None
    }
    train_spec.update(mlflow_spec)

data = {
    "action": "annotation",
    "specs": {
        "round_size": 2,  # round_size: number of images to annotate in each round, e.g. notify at least 2 different image_ids
        "stop_criteria": {
            "max_rounds": 2,
            "key_metric": 0.9,
        },
        "train_spec": train_spec,
    }
}

endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

assert response.status_code == 201, f"Run job failed, got {response.json()}."
cl_job_id = response.json()
print("Job creation succeeded with job ID: ", cl_job_id)

**Parameter Details**:
- `round_size`: Specifies how many new annotations are needed to trigger a new fine-tuning round for the model.
- `stop_criteria`: Criteria to decide when the continuous learning job should cease. 
    - `max_rounds`: Determines the maximum rounds the job should run.
    - `key_metric`: (Optional) If specified, the job will keep running until the designated evaluation metric reaches the value set.
- `train_spec`: Overrides certain parameters in the model for this particular training. If you have an MLflow server set up, you can add its  parameters under tracking to enable logging metrics with MLflow.

#### Check Job Status

Ensure the continuous learning job is up and running as expected:

In [ ]:
status = "Pending"
while status != "Running":
    endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{cl_job_id}"
    response = requests.get(endpoint, headers=headers)

    assert response.status_code == 200, f"Failed to get job status, got {response.json()}."
    status = response.json()["status"]
    print("Continuous Learning/Annotation Job status: ", status)
    time.sleep(1)

### Using MLflow to Monitor Metrics

If you've set up MLflow and included the relevant parameters in your continuous learning job, you can actively monitor the training metrics through the platform. This is invaluable for gauging the performance of your model in real-time and making timely interventions when necessary.

![Inference Auto Segmentation](./end2end_pic/mlflow.png)

## VISTA Workflows

Deep-dive into specific workflows that allow refined interaction with the model:

1. **Segment All Classes**: Users can analyze an entire image without specific prompts, offering a comprehensive overview.
2. **Using Class Prompts**: Users direct the model's focus towards one or more specific classes. Class-based segmentation can enable a specialized focus on a particular disease/organ.
3. **Using Point Prompts**: Users specify a sequence of background and foreground clicks to guide the model’s focus, particularly when used together with class prompts.

These workflows also integrate seamlessly with the OHIF Plugin for an enhanced visual experience, we'll walk through the OHIF experience below along with the accompanying API call used in the background.

### Using Segment Everything
By default, the VISTA-3D Experiment provides 118 classes and using the Auto Segmentation panel, you can run inferencing use all available classes.

**Steps**
1. Click the `run` button under the `Auto Segmentation panel` to obtain the segmentation mask for all classes.

![Inference Auto Segmentation](./end2end_pic/inference_as.png)

The associated API call run when you click the `Run` button is below:

In [ ]:
# get an inference image id with nextimage api
data = {
    "action": "nextimage"
}
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)

assert response.status_code == 201, f"Recommend image failed, got {response.json()}."
res = response.json()
inference_image_id = res["image"]
print(f"Recommended Image to annotate: {inference_image_id}")
print(json.dumps(res, indent=2))

data = {
    "action": "inference",
    "specs": {
        "image": inference_image_id,
        "bundle_params": {
            "label_prompt": list(range(1, 118))  # inference all 117 classes
        },
    }
}

endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Run inference failed, got {response.json()}."
print("Inference Successful.  Label is returned")
print(response.headers)

### Using Class Prompts
Instead of using all 118 labels, you can select a few labels that you're interested in and run inference only on those classes.  If you're using a customize version of VISTA-3D as referenced in our [Dataset Creation and Experiment Selection](./Dataset-Creation-and-Experiment-Selection.ipynb) notebook, you'll see only the classes you created with the model listed in this section.

**Steps**
 1. Click the `Class Prompts` panel.
 2. Select classes that you want to inference with class prompts.
 3. Click the `Run` button to get the inference result.

![Inference Point Prompts](./end2end_pic/inference_class_prompts.png)

After a few seconds, you will see the inference result.

![Inference Point Prompts Result](./end2end_pic/inference_class_prompts_res.png)

The associated API call run when you click the `Run` button is below:

In [ ]:
bundle_params = {
    "label_prompt": [1, 2, 3, 4, 5], # Whichever classes were selected
}

data = {
    "action": "inference",
    "specs": {
        "image": inference_image_id,
        "bundle_params": bundle_params,
    }
}

endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Run inference failed, got {response.json()}."
print("Inference Successful.  Label is returned")
print(response.headers)

### Using Point Prompts
Last, instead of using only class prompts, you can use point+class prompts.  This allows you to add points to the indicated classes to help guide the model and refine your segmentation using an interactive workflow.

**Steps**
 1. Click the `Point Prompts` panel.
 2. Select a class that you want to inference with point prompts.
 3. Add some point to the image where you want to get the mask by clicking.
 4. Click the `Run` button to get the inference result.

 ![Inference Point Prompts](./end2end_pic/inference_point.png)

After a few seconds, you will see the inference result.

 ![Inference Point Prompts Result](./end2end_pic/inference_point_res.png)

If you want to clear some points, you can either clear specific class points or clear all points by clicking the `Clear Points` or `Clear All Points` button.

![Clear Points](./end2end_pic/clearpoints.png)

The associated API call run when you click the `Run` button is below:

In [ ]:
bundle_params = {
    "points": [[20,20,20], [20, 40, 60]],
    "point_labels": [2, 2],
    "label_prompt": [2],
}

data = {
    "action": "inference",
    "specs": {
        "image": inference_image_id,
        "bundle_params": bundle_params,
    }
}

endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.post(endpoint, json=data, headers=headers)
assert response.status_code == 201, f"Run inference failed, got {response.json()}."
print("Inference Successful.  Label is returned")
print(response.headers)

## Annotation Workflow

Annotating medical images efficiently and precisely is a multi-step process. Here's a breakdown of the typical workflow you'd employ when using NVIDIA MONAI Cloud APIs and OHIF. We'll cover any relevant APIs not already covered as we walk through the workflow.

`Load Image` --> `Run Inference` --> `Annotate/Fix Annotation` --> `Save /Notify` --> `Repeat`

### 1. **Load Image**

Begin by loading the desired medical image that you wish to annotate. If you're using OHIF, you'll see the study list and can select a patient the annotate.  Make sure to use the `MONAI Service` to load the NVIDIA MONAI Cloud API plugin.

![Select an image](end2end_pic/selectanimage.png)

If you're using the API directly, you can use the `nextimage` endpoint.

### 2. **Run Inferencing Using Selected Method**

Choose one of the inferencing methods discussed above:

1. **Segment All Classes**
2. **Using Class Prompts**
3. **Using Point Prompts**

Once you've picked your preferred method, run the inference to get an initial annotation.

![allclass](./end2end_pic/allclassohif.png)

### 3. **Annotate / Refine Annotations**

With the initial mask in place, you might notice areas that require manual tweaking. Use the provided annotation tools to:

- Refine boundaries
- Add or remove regions

This step ensures that your annotations are as accurate as possible.

**Steps**
1. Click the Segmentation button.
2. Select a class of segmentation that needs to be updated.
3. Select a segmentation tool.
4. Update the segmentation with this tool.

![Annotate](./end2end_pic/annotate.png)

### 4. **Save and Notify the Server**

Once you're satisfied with your annotations, the first step is to save the annotated image, ensuring that your work is captured. This will write back the image using the DICOMWeb protocal back to your datastore.

![Save Label](./end2end_pic/savelabel.png)

Next, notify the server that an image has been annotated. This step is crucial for continuous learning. The system will take note of the new annotations and after the indicated number of annotated images it will use them to improve the model over time.

![Notify](end2end_pic/notify.png)

The associated API call run when you click the `Notify Server` button is below:

```python
# After uploading a DICOM Seg into DICOM Web
endpoint = f"{base_url}/datasets/{dataset_id}/jobs"
label_id = "<series_id_1>"
data = {
    "action": "notify"
    "specs": {
        "added": {
            "image": inference_image_id,
            "label": label_id,
        },
        "updated": [],
        "removed": [],
    }
}

response = requests.post(endpoint, json=data, headers=headers)
if response.status_code == 201:
    print("Notified.")
else:
    print(response.json())
    print(response)
```

### 5. **Repeat**

Continue the process for all the images in your dataset. With each iteration, not only do you expand your annotated dataset, but you also contribute to the model's learning, making future annotations even more accurate.

## Stopping a Continuous Learning Job

As your model refines itself over time using continuous learning, there might come a point where you need to halt the ongoing CL job. Whether you're satisfied with the model's performance or have other reasons, here's how you can stop the CL job:

In [ ]:
# Manually stop the CL job. No need to execute this cell if the job has reached the stop criteria.
endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{cl_job_id}:cancel"
response = requests.post(endpoint, headers=headers)
assert response.status_code == 200, f"cancel job failed, got {response.json()}."

## Stopping the experiment from Realtime Inference mode

When the experiment is created with `realtime_infer` as `True`, it will reserve one GPU to process the inference requests.

After we have finished the inference process, we would like to release the GPU resource for other tasks.

To achieve this, we can switch the `realtime_infer` from `True` to `False`.

Note: this step is irreversible, which means you can't set the `realtime_infer` from `False` to `True`. To bootstrap another inference, you will have to create another experiment.

In [ ]:
data = {
    "realtime_infer": False,
}

endpoint = f"{base_url}/experiments/{experiment_id}"
response = requests.patch(endpoint, json=data, headers=headers)
assert response.status_code == 200, f"stop job failed, got {response.json()}."

## <a id="Exporting-the-Experiment">Exporting the Experiment</a>

After you've trained the model, you might want to export it for various purposes.  Here's how you can accomplish that using the following APIs:

In [ ]:
# List all jobs and pick one job that meets your requirement.
endpoint = f"{base_url}/experiments/{experiment_id}/jobs"
response = requests.get(endpoint, headers=headers)

assert response.status_code == 200, f"List all jobs failed, got {response.json()}."
job_metas = response.json()
for job_meta in job_metas:
    if job_meta["id"] == cl_job_id:
        print("Continuous Learning Job status: ", job_meta["status"])
    else:
        train_id = job_meta["id"]
        print(f"Training Job {train_id} status: ", job_meta["status"])
        if job_meta["status"] == "Done":
            print(f"Training Job {train_id} with ID {job_meta['id']} metric: ", job_meta["result"]["key_metric"])

In [ ]:
# Pick a job id from the last cell output. For example, choose train_id
if "train_id" in locals():
    download_job_id = train_id
    endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{download_job_id}:download"
    response = requests.get(endpoint, data=json.dumps({"export_type": "monai_bundle"}), headers=headers)
    assert response.status_code == 200, f"Failed to export job, got {response.json()}."
    with open(f"{download_job_id}.tar.gz", "wb") as fp:
        fp.write(response.content)
    print("Downloaded!")

### Detailed Logging Through Download API

For a more comprehensive view and detailed logging of your jobs, our platform offers a Download API. This API enables you to access in-depth logs, model checkpoints, and data outputs, which are instrumental for troubleshooting, in-depth analysis, and gaining insights into the specifics of your job's execution. The Download API is particularly useful if your job encounters an error or if you need to understand the performance and behavior of your job in greater detail.

In [ ]:
if "download_job_id" in locals():
    endpoint = f"{base_url}/experiments/{experiment_id}/jobs/{download_job_id}:download"
    response = requests.get(endpoint, headers=headers)

    assert response.status_code == 200, f"Failed to download job, got {response.json()}."
    #save to file
    attachment_data = response.content
    with open(f"{download_job_id}.tar.gz", 'wb') as f:
        f.write(attachment_data)
    print(f"Bundle training results are downloaded as {download_job_id}.tar.gz")

## Conclusion

Remember, the NVIDIA MONAI Cloud APIs are designed to streamline this process, making it intuitive and efficient. As you work through these steps, the platform aids you, ensuring that you can focus on the quality of annotations while the technical details are handled seamlessly in the background.

Make the most of continuous learning and annotation with the NVIDIA MONAI Cloud APIs. This iterative refinement paves the way to excellence in medical imaging.